### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

/home/miguel/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## step1 : Load and split the dataset
loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)


In [3]:
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using st

In [4]:
### step 2: Vector Store
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

## step 3:MMR Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7c42ad79fe90>, search_type='mmr', search_kwargs={'k': 5})

In [5]:
## step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model("openai:o4-mini")
llm


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c421dd94550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7c421dbe43d0>, root_client=<openai.OpenAI object at 0x7c421df73d50>, root_async_client=<openai.AsyncOpenAI object at 0x7c421dbe40d0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [6]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.

Original query: "{query}"

Expanded query:
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\n\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c421dd94550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7c421dbe43d0>, root_client=<openai.OpenAI object at 0x7c421df73d50>, root_async_client=<openai.AsyncOpenAI object at 0x7c421dbe40d0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [7]:
query_expansion_chain.invoke({"query":"Langchain memory"})

'“LangChain memory” OR “Lang Chain memory management” OR “Langchain conversational memory” OR “persistent context storage” OR “stateful agent memory” OR “session storage for LLMs” OR “memory modules” OR “memory backends” OR “ConversationBufferMemory” OR “ConversationSummaryMemory” OR “EntityMemory” OR “CombinedMemory” OR “memory store” OR “vector memory store” OR “Redis memory” OR “SQLite memory” OR “in-memory cache” OR “retrieval-augmented generation” OR “chat history retrieval” OR “context window management” OR “LLM memory engineering” OR “Python LangChain memory examples”'

In [8]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=answer_prompt)

In [9]:
# Step 5: Full RAG pipeline with query expansion
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [10]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query:
“(LangChain OR “Lang Chain”) AND (“memory support” OR memory OR “memory module” OR “conversational memory” OR “session memory” OR context OR “state management” OR “context persistence”) AND (types OR categories OR implementations OR modules OR backends OR patterns OR architectures) AND (examples OR e.g. OR such as OR including) AND (ConversationBufferMemory OR ConversationSummaryMemory OR CombinedMemory OR VectorStoreRetrieverMemory OR “external vector store” OR “knowledge base” OR Redis OR PostgreSQL OR Pinecone OR Chroma OR Milvus OR Weaviate) AND (ephemeral OR persistent OR summary OR buffer OR retrieval‐augmented)
✅ Answer:
 LangChain currently ships two “plug-and-play” memory back-ends for chat-based agents:

1. ConversationBufferMemory  
   – Keeps a running buffer of the full back-and-forth.  
2. ConversationSummaryMemory  
   – Actively summarizes older turns into a concise recap to stay within token limits.


In [11]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query:

("CrewAI agents" OR "Crew AI agents" OR "autonomous crew management agents" OR "AI-based crew coordination agents" OR "virtual crew assistants" OR "autonomous crew assistants" OR "crew management software agents" OR "AI crew scheduler" OR "mult i-agent system for crew planning" OR "agent-based modeling for crew assignment" OR "reinforcement learning crew scheduler" OR "distributed crew coordination agents")  
AND  
("crew scheduling" OR "workforce management" OR "staff rostering" OR "resource allocation" OR "operations planning")  
AND  
("airline crew" OR "maritime crew" OR "railway staff" OR "hospital nursing staff" OR "logistics teams")  
AND  
("machine learning" OR "multi-agent systems (MAS)" OR "reinforcement learning" OR "natural language processing" OR "decentralized AI" OR "software agents architecture")
✅ Answer:
 CrewAI agents are semi-autonomous, role-based workers in a multi-agent system.  Key characteristics include:  
• Defined Role – e.g. researcher, pl